In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
main_df = pd.read_csv("../data/raw/initial_games_data.csv")
platform_df = pd.read_csv("../data/raw/platform_game_data.csv")
steam_games_df = pd.read_csv("../data/raw/games.csv", index_col=False)

In [ ]:
print(f"Main Dataset: {main_df.shape[0]} Second dataset: {platform_df.shape[0]}")

In [ ]:
#Picking the attributes to keep for dataset 3
steam_games_df = steam_games_df[["AppID", "Categories", "Genres", "Tags"]]
steam_games_df = steam_games_df.rename(columns={"AppID": "app_id"})

In [ ]:
main_df = main_df.drop(columns=["Unnamed: 0"]) # Drop unnecessary index column
main_df = main_df.iloc[:, :4] # Keep only the first 4 columns


In [ ]:
# Keeps the first occurrence of each game via link
main_df = main_df.drop_duplicates(subset='link', keep='first')

In [ ]:
main_df["app_id"] = main_df["link"].str.extract(r'/app/(\d+)/')
main_df['app_id'] = main_df['app_id'].astype(int) 

In [ ]:
main_df = main_df.drop(columns=['link'])

In [ ]:
cols = ['app_id'] + [c for c in main_df.columns if c != 'app_id']
main_df = main_df[cols]

In [ ]:
platform_df = platform_df.drop(columns=["rating", "positive_ratio", "user_reviews", "price_final", "discount", "steam_deck"])

In [ ]:
merged_df = pd.merge(main_df, platform_df, on='app_id')

In [ ]:
differences = merged_df[merged_df["release"] != merged_df["date_release"]]

In [ ]:
game_dupes = merged_df["game"].duplicated().sum()
title_dupes = merged_df["title"].duplicated().sum()

In [ ]:

merged_df = merged_df.drop(columns=["game", "date_release"])


merged_final = pd.merge(
    merged_df,
    steam_games_df,
    on="app_id",
    how="inner" 
)

# print(merged_final.info())
print(merged_final.head(10))

matched = merged_final["Categories"].notna().sum()
print(f"Games with category info added: {matched}")
